In [1]:
%matplotlib widget

import os
import pathlib
import pprint

from Visualizer.DataSet import DataSet
from Visualizer.Monitoring.Persistence.ProfilerSession import ProfilerSession
from Visualizer.Plotting.Plot import Plot

# [Analyzer] Single Profiler Session

This notebook can visualize a specific profiler session.

## Configuration

Configure the project settings below.
The default configuration should be okay.
Make sure that you have run at least one profiler session with the EnergyManager and have generated a database, otherwise this notebook will fail.

In [61]:
database = f"{os.environ.get('HOME')}/EnergyManager-Resources/Database.sqlite"

# Load all profiler sessions
complete_data_set = DataSet(ProfilerSession.load_all(database))

### Profiler Sessions Overview

Below all profiler sessions are shown.

In [62]:
# Visualize the profiler session data
overview_plot = complete_data_set.table_plot
overview_plot.maximum_rows = 500
overview_plot.plot()

,ID,Label,Profile
0,1,Matrix Multiply,"{'core': 0, 'gpu': 0, 'iterations': 50, 'matrixAHeight': 1536, 'matrixAWidth': 1536, 'matrixBHeight': 1536, 'matrixBWidth': 1536}"
1,2,Matrix Multiply (EnergyMonitor MinMax),"{'core': 0, 'gpu': 0, 'iterations': 50, 'matrixAHeight': 1536, 'matrixAWidth': 1536, 'matrixBHeight': 1536, 'matrixBWidth': 1536, 'maximumCPUClockRate': 2601000000.0}"
2,3,Matrix Multiply,"{'core': 0, 'gpu': 0, 'iterations': 50, 'matrixAHeight': 1536, 'matrixAWidth': 1536, 'matrixBHeight': 1536, 'matrixBWidth': 1536}"
3,4,Matrix Multiply (EnergyMonitor MinMax),"{'core': 0, 'gpu': 0, 'iterations': 50, 'matrixAHeight': 1536, 'matrixAWidth': 1536, 'matrixBHeight': 1536, 'matrixBWidth': 1536, 'maximumCPUClockRate': 2601000000.0}"
4,5,Matrix Multiply (EnergyMonitor System),"{'core': 0, 'gpu': 0, 'iterations': 50, 'matrixAHeight': 1536, 'matrixAWidth': 1536, 'matrixBHeight': 1536, 'matrixBWidth': 1536, 'maximumCPUClockRate': 2601000000.0}"


## Analysis

Define the ID of the test you wish to visualize below.

In [73]:
#profiler_session_id = complete_data_set.data[-1].id
profiler_session_id = 3
data_set = DataSet([profiler_session for profiler_session in complete_data_set.data if profiler_session.id == profiler_session_id])

### Summary

These are the specifications of the system with which the profiler session was performed.

In [5]:
pprint.pprint(data_set.data[0].summary)

{'GPUs': [{'Brand': 'GeForce',
           'Compute Capability Major Version': 7,
           'Compute Capability Minor Version': 5,
           'Default Auto Boosted Clocks Enabled': None,
           'Default Power Limit (W)': '260.0',
           'Memory Size (B)': '8.3 GB',
           'Name': 'GeForce RTX 2080',
           'PCIe Link Width (B)': '16 Bytes',
           'Supported Core Clock Rates (Hz)': [''],
           'Supported Memory Clock Rates (Hz)': ['']}],
 'Label': 'Fixed Frequency Jacobi'}


### Profile

This is the profile with which the profiler session was performed.

In [6]:
pprint.pprint(data_set.data[0].profile)

{'core': 0,
 'domainHeight': 1000,
 'domainWidth': 1000,
 'gpu': 0,
 'iterations': 15,
 'maximumCPUClockRate': 2044444444.444444,
 'maximumGPUClockRate': 2550000000.0,
 'minimumCPUClockRate': 2044444444.444444,
 'minimumGPUClockRate': 2550000000.0,
 'topologyHeight': 1,
 'topologyWidth': 1}


### Clock Rates

The graph below shows the clock rate over time.

In [75]:
clock_rate_timeseries_plot = data_set.data[0].clock_rate_timeseries_plot(plot_limits=False, use_ear=False)
clock_rate_timeseries_plot.plot_series = { key: clock_rate_timeseries_plot.plot_series[key] for key in clock_rate_timeseries_plot.plot_series if (key.startswith("EAR") or key.startswith("CPU 0") and (not key.startswith("CPU 0 Core") or key.startswith("CPU 0 Core 0"))) or key.startswith("GPU 0")}
clock_rate_timeseries_plot.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Correlations

The graphs below shows the Pearson correlation heatmap for each performance variable.

In [8]:
#data_set.data[0].monitor_data_correlations_plot.plot()

### Energy Consumption

The graphs below shows the total energy consumption over time.

In [67]:
# Plot in Joules
energy_consumption_plot_j = data_set.data[0].energy_consumption_timeseries_plot(use_ear=False)
energy_consumption_plot_j.plot_series = { key: energy_consumption_plot_j.plot_series[key] for key in energy_consumption_plot_j.plot_series if (key.startswith("EAR") or key.startswith("CPU 0") and (not key.startswith("CPU 0 Core") or key.startswith("CPU 0 Core 0"))) or key.startswith("GPU 0") or key.startswith("Node")}
energy_consumption_plot_j.plot()

# Plot in Watt Hours
energy_consumption_plot_wh = data_set.data[0].energy_consumption_timeseries_plot(unit_string="Wh", modifier=Plot.j_to_wh, use_ear=False)
energy_consumption_plot_wh.plot_series = { key: energy_consumption_plot_wh.plot_series[key] for key in energy_consumption_plot_wh.plot_series if (key.startswith("EAR") or key.startswith("CPU 0") and (not key.startswith("CPU 0 Core") or key.startswith("CPU 0 Core 0"))) or key.startswith("GPU 0") or key.startswith("Node")}
energy_consumption_plot_wh.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Events

The graph below shows the events over time.

In [10]:
for events_plot in data_set.data[0].events_event_plot()[:1]:
    events_plot.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Fan Speed

The graphs below shows the fan speed over time.

In [11]:
data_set.data[0].fan_speed_timeseries_plot.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Instructions Per Cycle

The graph below shows the utilization rates over time.

In [12]:
data_set.data[0].instructions_per_cycle_timeseries_plot.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.


### Memory Consumption

The graphs below shows the memory consumption over time.

In [13]:
data_set.data[0].memory_consumption_timeseries_plot(plot_sizes=False).plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Power Consumption

The graphs below shows the power consumption over time.

In [14]:
power_consumption_timeseries_plot = data_set.data[0].power_consumption_timeseries_plot(plot_limits=False, use_ear=False)
#power_consumption_timeseries_plot.plot_series = { key: power_consumption_timeseries_plot.plot_series[key] for key in power_consumption_timeseries_plot.plot_series if (key.startswith("CPU 1") or key.startswith("EAR") or key.startswith("CPU 0") and (not key.startswith("CPU 0 Core") or key.startswith("CPU 0 Core 0"))) or key.startswith("GPU 0") or key.startswith("Node")}
power_consumption_timeseries_plot.plot_series = { key: power_consumption_timeseries_plot.plot_series[key] for key in power_consumption_timeseries_plot.plot_series if (key.startswith("EAR") or key.startswith("CPU 0") and (not key.startswith("CPU 0 Core") or key.startswith("CPU 0 Core 0"))) or key.startswith("GPU 0") or key.startswith("Node")}
power_consumption_timeseries_plot.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Processes

The graphs below shows the amount of processes over time.

In [15]:
data_set.data[0].processes_timeseries_plot.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### States

The graph below shows the states of the EnergyMonitor over time.

In [16]:
data_set.data[0].states_event_plot().plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Temperature

The graphs below shows the temperature over time.

In [17]:
temperature_timeseries_plot = data_set.data[0].temperature_timeseries_plot
temperature_timeseries_plot.plot_series = { key: temperature_timeseries_plot.plot_series[key] for key in temperature_timeseries_plot.plot_series if (key.startswith("CPU 0") and (not key.startswith("CPU 0 Core") or key.startswith("CPU 0 Core 0"))) or key.startswith("GPU 0") or key.startswith("Node")}
temperature_timeseries_plot.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Timespan

The graphs below shows the time spent on different types of operations over time.

In [71]:
timespan_timeseries_plot = data_set.data[0].timespan_timeseries_plot(use_ear=False)
timespan_timeseries_plot.plot_series = { key: timespan_timeseries_plot.plot_series[key] for key in timespan_timeseries_plot.plot_series if (key.startswith("Runtime") or key.startswith("EAR") or key.startswith("CPU 0 Core 0"))}
timespan_timeseries_plot.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Utilization Rates

The graph below shows the utilization rates over time.

In [19]:
utilization_rate_timeseries_plot = data_set.data[0].utilization_rate_timeseries_plot
utilization_rate_timeseries_plot.plot_series = { key: utilization_rate_timeseries_plot.plot_series[key] for key in utilization_rate_timeseries_plot.plot_series if (key.startswith("CPU 0") and (not key.startswith("CPU 0 Core") or key.startswith("CPU 0 Core 0"))) or key.startswith("GPU 0") or key.startswith("Node")}
utilization_rate_timeseries_plot.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …